In [1]:
import pandas as pd
import pymysql
import logging
import sshtunnel
from sshtunnel import SSHTunnelForwarder
import os.path

In [2]:
# You will need to add your SSH public key and database password to connect.
ssh_host = 'vfsmghnecweb01.fsm.northwestern.edu'
ssh_username = 'efa3001'
db_server = 'vfsmghnecdb01.fsm.northwestern.edu'
database_username = 'efa3001'
ssh_pkey = 'C:\\Users\\16262\\Desktop\\NCDS-NEC_Data_Internship\\5-16-privkey-openSSH'
database_password = 'E%H2)h.nPevc<a>&'
database_name = 'nec_dw'
localhost = '127.0.0.1'

In [3]:
def open_ssh_tunnel(verbose=False):
    """Open an SSH tunnel and connect using a username and password.
    
    :param verbose: Set to True to show logging
    :return tunnel: Global SSH tunnel connection
    """
    
    if verbose:
        sshtunnel.DEFAULT_LOGLEVEL = logging.DEBUG
    
    global tunnel
    tunnel = SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_pkey = ssh_pkey,
        remote_bind_address = (db_server, 3306)
    )
    
    tunnel.start()

In [4]:
def mysql_connect():
    """Connect to a MySQL server using the SSH tunnel connection
    
    :return connection: Global MySQL database connection
    """
    
    global connection
    
    connection = pymysql.connect(
        host = localhost,
        user = database_username,
        passwd = database_password,
        db = database_name,
        port = tunnel.local_bind_port
    )

In [5]:
def run_query(sql):
    """Runs a given SQL query via the global database connection.
    
    :param sql: MySQL query
    :return: Pandas dataframe containing results
    """
    
    return pd.read_sql_query(sql, connection)

In [6]:
def mysql_disconnect():
    """Closes the MySQL database connection.
    """
    
    connection.close()

In [7]:
def close_ssh_tunnel():
    """Closes the SSH tunnel connection.
    """
    
    tunnel.close

In [8]:
open_ssh_tunnel()

In [9]:
mysql_connect()

In [10]:
df = run_query("SELECT * FROM Reports LIMIT 10")
df.head()

,ReportID,ProjectID,Title,Status,ApproachesAndInterventionsUsed,ContinuationPlans,Evaluation,Impact,LessonsLearned,ProblemsOrBarriersEncountered,Other,ModifiedDate
0,17168,17166,Final Report: Opportunities for Health Educati...,Accept,<p>The Oklahoma Healthy Aging Initiative (OHAI...,<p>All of the activities listed will continue ...,<p>OHAI main project goals was to provide comm...,The funding provided allowed for many people t...,<p>The most unexpected result that OHAI experi...,<p>OHAI only discovered two major barriers or ...,,2020-05-27
1,17151,17149,Final Report: MACMLA Annual Conference- 2019 R...,Save Draft,,,,,,,,2019-02-02
2,17141,17139,Final Report: NNLM All of Us Community Engagem...,Accept,<p>In collaboration with strategic partners in...,<p>We will continue to have health information...,<p>We utilized project outcome surveys for the...,The impact on our project is still being expl...,<p>We learned that people are extremely intere...,<p>Our public health and health sciences libra...,,2020-05-19
3,17138,17136,Final Report: NNLM All of Us Community Engagem...,Accept,"<p>Outreach Librarian, LaVentra Danquah, condu...",<p>Shiffman Medical Library plans to continue ...,<p>Activity reports for activities and events ...,This project contributed toward Shiffman Medic...,<p>A lesson learned from the multiple exhibit ...,<p>A limited number of staff working on the NN...,,2020-05-19
4,17135,17133,Final Report: NNLM All of Us Community Engagem...,Submit for Review,,,,,,,,2020-06-19


In [11]:
mysql_disconnect()
close_ssh_tunnel()

In [13]:
# Write the data frame to a tab-seperated text file
import os  

file_path = "C:\\Users\\16262\\Desktop\\nec-internship-project"

if not os.path.exists(file_path):
    os.mkdir(file_path)

output_file = 'NNLM_final_reports_sample.tsv'
complete_fn = os.path.join(file_path, output_file)
df.to_csv(complete_fn, sep = '\t',index=False)